# 1. Boîte à outils

In [ ]:
import Toolbox as tb

# 2. Diffie-Hellman

Implémenter une communication entre Arielle et Bertrand sous TCP (on
pourra considérer que l'un des deux est un serveur et l'autre un
client) permettant d'appliquer le protocole de Diffie-Hellman. Faites
afficher à chaque participant les étapes du protocole sur sa sortie
standard.

In [ ]:
import User as usr
p, g = 8779, 4157
b = 3242
a = 2682

port = 1024
usr.bertrand(port,g,p,b)
usr.arielle(port,g,p,a)

Bertrand : initialized on port 1024Arielle 35984 : connected
Bertrand : Listening
Bertrand : Got new connection from 127.0.0.1 at 35984

Arielle : Sending 0 to 1024
Bertrand : Received 0 from 35984
Bertrand : Sending 0 to 35984
Bertrand : calculated key : 0Arielle : Received 0 from 1024
Arielle : calculated key : 0
Arielle : Disconnected

Bertrand : Closing the connection from 127.0.0.1 at 35984
Bertrand : Disconnected


# 3. L'homme du milieu

Arielle et Bertrand veulent communiquer en utilisant RSA. Implémenter
le serveur de Laurent pour qu'il se fasse passer pour Bertrand auprès
d'Arielle (et inversement). Laurent affichera sur la sortie standard
les messages échangés par Arielle et Bertrand (sous forme déchiffrée).

In [ ]:
import threading
import socket as sckt
import Toolbox

def arielle(port):
  def arielle_behavior(port):
    # connexion à Laurent
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    socket.bind(("127.0.0.1", port))
    socket.listen(True)
    print("Arielle : initialized on port {}".format(port))
    print("Arielle : Listening")
    conn, addr = socket.accept()
    print('Arielle : Got new connection from {} at {}'.format(addr[0], addr[1]))
    
    # génération des clés + échange des clés publiques

    pub_key = 0
    priv_key = 0

    pub_key1 = 0

    print("Arielle : Sending {} to {}".format(pub_key, addr[1]))
    conn.send(str(pub_key).encode())

    pub_key1 = int(conn.recv(port).decode())
    print("Arielle : Received {} from {}".format(pub_key1, addr[1]))

    # communication

    mess = int(conn.recv(port).decode())
    print("Arielle : Received {} from {}".format(mess, addr[1]))
    

    # Fin
    socket.listen(False)
    socket.close()
    print("Arielle : Disconnected")


    # exemple
    # # Arielle attend qu'on lui envoie
    # mess = int(conn.recv(port).decode())
    # print("Arielle : Received {} from {}".format(mess, addr[1]))
    
    # # Arielle recoit
    # mess = ""
    # print("Arielle : Sending {} to {}".format(mess, addr[1]))
    # conn.send(str(mess).encode())

  x = threading.Thread(target=arielle_behavior, args=([port]))
  x.start()
    
def bertrand(port):
  def bertrand_behavior(port):
    # connexion à Bertrand
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    socket.bind(("127.0.0.1", port))
    socket.listen(True)
    print("Bertrand : initialized on port {}".format(port))
    print("Bertrand : Listening")
    conn, addr = socket.accept()
    print('Bertrand : Got new connection from {} at {}'.format(addr[0], addr[1]))
    
    # génération des clés + échange des clés publiques

    pub_key = 0
    priv_key = 0

    pub_key1 = 0

    print("Bertrand : Sending {} to {}".format(pub_key, addr[1]))
    conn.send(str(pub_key).encode())
    pub_key1 = int(conn.recv(port).decode())
    print("Bertrand : Received {} from {}".format(pub_key1, addr[1]))

    # communication

    mess = "test"


    print("Bertrand : Sending {} to {}".format(mess, addr[1]))
    conn.send(str(mess).encode())

    # Fin
    socket.listen(False)
    socket.close()
    print("Bertrand : Disconnected")


    # exemple
    # # Bertrand attend qu'on lui envoie
    # mess = int(conn.recv(port).decode())
    # print("Bertrand : Received {} from {}".format(mess, addr[1]))
    
    # # Bertrand recoit
    # mess = ""
    # print("Bertrand : Sending {} to {}".format(mess, addr[1]))
    # conn.send(str(mess).encode())

  x = threading.Thread(target=bertrand_behavior, args=([port]))
  x.start()


def laurent(port1,port2):
  # connexion à Arielle et Bertrand
  # Arielle
  socket1 = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  try :
    socket1.connect(("127.0.0.1", port1))
    print("Laurent {} : connected".format(socket1.getsockname()[1]))
  except :
    print("Laurent : Connection Error")
    raise Exception
  
  # Bertrand
  socket2 = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  try :
    socket2.connect(("127.0.0.1", port2))
    print("Laurent {} : connected".format(socket2.getsockname()[1]))
  except :
    print("Laurent : Connection Error")
    raise Exception

  # génération des clés + échange des clés publiques
  
  pub_key = 0
  priv_key = 0

  pub_key1 = 0
  pub_key2 = 0

  pub_key1 = int(socket1.recv(port1).decode())
  print("Laurent : Received {} from {}".format(pub_key1, port1))

  pub_key2 = int(socket2.recv(port2).decode())
  print("Laurent : Received {} from {}".format(pub_key2, port2))
  
  socket1.send(str(pub_key).encode())
  socket2.send(str(pub_key).encode())
  
  # affichage des messages interceptés

  mess = int(socket2.recv(port2).decode())


  blabla mess

  socket1.send(str(mess).encode())



  

# 4. Malléabilité

Implémenter Laurent pour que, lorsqu'il intercepte un message chiffré
`c` correspondant au message `m` (qu'il ne peut retrouver), il
transmette à l'autre participant un message `c'` correspondant au
message `2m`.

In [ ]:
def arielle():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # envoi de messages chiffrés (+ affichage)
    return
    
def bertrand():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # déchiffrement des messages reçus (+ affichage)
    return
    
def laurent():
    # connexion à Arielle et Bertrand
    
    # transmission de la clé d'Arielle à Bertrand (et inversement)
   
    # modification et transmission des messages interceptés
    return